This function checks the orientation of the vector field and computes the local (linear) interpolant of the vector-field.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| x | array (2,) | tensorline position $ \mathbf{x} $ |
| x_prime | array (2,) | eigenvector at point $ \mathbf{x} $ (=tangent to tensorline)|
| vector_field | array(Ny, Nx) | eigenvector field (not oriented)|

In [ ]:
# Import numpy
import numpy as np

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [ ]:
def _orient_vectorfield(X, Y, x, vector_field):
            
        # Check location of particles with respect to the indices of the meshgrid
        idx_x = np.searchsorted(X[0,:], x[0])
        idx_y = np.searchsorted(Y[:,0], x[1])
        
        # If idx_x, idx_y is within indices of meshgrid
        if 0 < idx_x < X.shape[1] and 0 < idx_y < Y.shape[0]:
    
            # compute reduced meshgrid around point
            X_reduced, Y_reduced = X[idx_y-1:idx_y+1, idx_x-1:idx_x+1], Y[idx_y-1:idx_y+1, idx_x-1:idx_x+1]
            vx_grid = np.array([[vector_field[idx_y-1,idx_x-1, 0], vector_field[idx_y, idx_x-1, 0]],
                      [vector_field[idx_y-1,idx_x, 0], vector_field[idx_y, idx_x, 0]]])
            vy_grid = np.array([[vector_field[idx_y-1,idx_x-1, 1], vector_field[idx_y, idx_x-1, 1]],
                      [vector_field[idx_y-1,idx_x, 1], vector_field[idx_y, idx_x, 1]]])
        
            # re-orient velocities in the reduced meshgrid
            for i in range(2):
                for j in range(2):
                    if vx_grid[0, 0]*vx_grid[i, j]+vy_grid[0, 0]*vy_grid[i, j] < 0:
                        vx_grid[i, j] = -vx_grid[i, j]
                        vy_grid[i, j] = -vy_grid[i, j]
    
            # linearly interpolate the (re-oriented) velocity within the reduced meshgrid
            vx_Interp = RBS(Y_reduced[:,0], X_reduced[0,:], vx_grid, kx = 1, ky = 1)
            vy_Interp = RBS(Y_reduced[:,0], X_reduced[0,:], vy_grid, kx = 1, ky = 1)
        
            # evalute interpolated (re-oriented) velocity
            return vx_Interp(x[1], x[0])[0][0], vy_Interp(x[1], x[0])[0][0]
        
        else:
            
            return None, None